In [ ]:
### Installs/imports
#!pip install torch transformers datasets tabulate scikit-learn seaborn accelerate bitsandbytes
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import json
import os
import random
import torch.nn.functional as F
from collections import defaultdict
from enhanced_hooking import get_activations, add_activations_and_generate, clear_hooks, get_activations_and_generate, zeroout_projections_and_generate
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns
from datasets import load_dataset
from enum import Enum
class SteeringType(Enum):
    IN_PROMPT = "In prompt"
    CONTINUOUS = "Continuous"
class AggType(Enum):
    MEANDIFF = "MeanDiff"
    PCA = "PCA"

%load_ext autoreload
%autoreload 2

In [ ]:
#### Load the model

HF_TOKEN = os.getenv('HF_TOKEN')

model=None
import gc
gc.collect()
torch.cuda.empty_cache()

base_model_path: str = "meta-llama/Llama-2-13b-chat-hf"
model_path=base_model_path
###model_path="cackerman/llama2_13b_chat_projection_tune_neg_in"

from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
_ = torch.set_grad_enabled(False)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, token=HF_TOKEN, quantization_config=bnb_config, device_map="auto")
device = model.device
tokenizer = AutoTokenizer.from_pretrained(base_model_path, token=HF_TOKEN)
model.tokenizer = tokenizer
if model.tokenizer.pad_token is None:
    new_pad_token = model.tokenizer.eos_token
    num_added_tokens = model.tokenizer.add_special_tokens({'pad_token': new_pad_token})
    model.resize_token_embeddings(len(model.tokenizer))
    model.config.pad_token_id = model.tokenizer.pad_token_id
model_numlayers = model.config.num_hidden_layers

In [ ]:
### Load in data

outputdir = "../output/"
datadir = "../data/"

user_tag, asst_tag = "[INST] ", " [/INST]"
sys_tag_open, sys_tag_closed = "<<SYS>> ", " <</SYS>>"
model_numlayers = model.config.num_hidden_layers

key_token_offset = 1###2
pair_statement=False # pair labels
test_on_gpt4facts=True

strip_quotes = lambda s: s[1:-1] if len(s) > 1 and (s.startswith('"') and s.endswith('"') or s.startswith("'") and s.endswith("'")) else s

### Contrastive prompts used to identify vectors ######
pos_persona = "Pretend that you are always honest, no matter what."
neg_persona = "Pretend that you are dishonest when it serves you."
pos_label = "A"
neg_label = "B"

df = pd.read_csv(datadir+"facts_true_false.csv") #train
true_statements = df[df['label'] == 1]['statement'].values.tolist()
false_statements = df[df['label'] == 0]['statement'].values.tolist()

def template(persona: str, stmt: str, label: str) -> str:
    return f"{user_tag}{sys_tag_open}{persona}{sys_tag_closed} Question: Is the following claim true or false? \"{strip_quotes(stmt)}\"\n\nChoices:\n A) True\n B) False\n\nAnswer:{asst_tag} {label}"

learn_directions_dataset = []
if pair_statement:
    for truth in true_statements:
        learn_directions_dataset.append((template(pos_persona, truth, pos_label), template(neg_persona, truth, neg_label)))
    for lie in false_statements:
        learn_directions_dataset.append((template(pos_persona, lie, neg_label), template(neg_persona, lie, pos_label)))

    letters_pos = np.array(["T" for _ in range(len(learn_directions_dataset)//2)] + ["F" for _ in range(len(learn_directions_dataset)//2)])
    letters_neg = np.array(["F" for _ in range(len(learn_directions_dataset)//2)] + ["T" for _ in range(len(learn_directions_dataset)//2)])

else:
    for i, (truth, lie) in enumerate(zip(true_statements, false_statements)):#honest response always comes first in pair
        learn_directions_dataset.append((template(pos_persona, truth, pos_label), template(neg_persona, lie, pos_label)))
    for i, (truth, lie) in enumerate(zip(true_statements, false_statements)):# just to keep the letters ordering consistent
        learn_directions_dataset.append((template(pos_persona, lie, neg_label), template(neg_persona, truth, neg_label)))

    letters_pos = np.array(["T" for _ in range(len(learn_directions_dataset)//2)] + ["F" for _ in range(len(learn_directions_dataset)//2)])
    letters_neg = np.array(["T" for _ in range(len(learn_directions_dataset)//2)] + ["F" for _ in range(len(learn_directions_dataset)//2)])

print(len(learn_directions_dataset))
for i in range(2):
    print(f"learn_directions_dataset[{i}]:", *learn_directions_dataset[i])
for i in range(len(learn_directions_dataset)-2, len(learn_directions_dataset)):
    print(f"learn_directions_dataset[{i}]:", *learn_directions_dataset[i])
###################################################################

In [ ]:
### get indexes where unsteered model gets it right

cor_indexes = []
sampling_kwargs={"use_cache": True, "pad_token_id": model.tokenizer.eos_token_id, "max_new_tokens": 1, "do_sample": False}
model.tokenizer.padding_side = "left"
clear_hooks(model)
model.eval()

for i, (true_claim, false_claim) in enumerate(tqdm(learn_directions_dataset)):
    if i == len(learn_directions_dataset)//2: break
    input_text = true_claim[:-1].replace(f"{sys_tag_closed}",f" Output only the letter corresponding to your choice.{sys_tag_closed}")

    inputs = model.tokenizer(input_text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    generated_tokens = model.generate(**inputs, **sampling_kwargs)
    true_output = model.tokenizer.decode(generated_tokens[0][len(inputs['input_ids'][0]):],skip_special_tokens=True)

    if true_output.lower() != "a": continue

    input_text = false_claim[:-1].replace(f"{sys_tag_closed}",f" Output only the letter corresponding to your choice.{sys_tag_closed}")

    inputs = model.tokenizer(input_text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    generated_tokens = model.generate(**inputs, **sampling_kwargs)
    false_output = model.tokenizer.decode(generated_tokens[0][len(inputs['input_ids'][0]):],skip_special_tokens=True)
    
    if false_output.lower() in ["b"]: cor_indexes.append(i)

print(cor_indexes)

In [ ]:
cor_indexes = [0, 1, 4, 5, 6, 7, 8, 9, 11, 12, 14, 19, 21, 23, 24, 26, 28, 30, 33, 34, 38, 39, 41, 43, 45, 47, 48, 50, 51, 52, 53, 56, 57, 63, 64, 66, 67, 68, 69, 72, 73, 74, 76, 77, 78, 79, 80, 81, 83, 84, 88, 91, 92, 96, 97, 98, 100, 101, 105, 106, 108, 109, 111, 112, 114, 116, 118, 121, 122, 123, 124, 125, 128, 129, 130, 131, 133, 135, 138, 139, 141, 142, 143, 144, 145, 146, 150, 152, 153, 157, 158, 161, 162, 164, 165, 175, 179, 180, 182, 184, 185, 186, 188, 189, 190, 191, 194, 195, 197, 198, 199, 203, 205, 206, 207, 208, 210, 211, 212, 214, 216, 218, 221, 223, 225, 227, 228, 232, 233, 235, 236, 237, 238, 239, 240, 242, 243, 244, 245, 247, 248, 249, 250, 251, 253, 257, 258, 259, 261, 265, 267, 270, 272, 275, 276, 277, 278, 279, 280, 281, 282, 286, 287, 288, 290, 295, 296, 298, 299, 301, 304]


In [ ]:
### Run contrastive prompts through model and capture activations

steering_type = SteeringType.CONTINUOUS
model.tokenizer.padding_side = "right"
token_offsets = [0]
key_token_offset = 1
layers = range(model_numlayers)

accumulated_activations_diffs = defaultdict(lambda: defaultdict(lambda: torch.empty(0)))
accumulated_activations_pos = defaultdict(lambda: defaultdict(lambda: torch.empty(0)))
accumulated_activations_neg = defaultdict(lambda: defaultdict(lambda: torch.empty(0)))

cor_indexes.extend([i + len(learn_directions_dataset)//2 for i in cor_indexes])

learn_directions_dataset_cor = [learn_directions_dataset[i] for i in cor_indexes]
learn_dataset = learn_directions_dataset_cor

batch_size = 2
batched_dataset = [
(
    [pair[0] for pair in learn_dataset[i:i + batch_size]],  # batch_pos
    [pair[1] for pair in learn_dataset[i:i + batch_size]]  # batch_neg
)
for i in range(0, len(learn_dataset), batch_size)
]

for batch_pos, batch_neg in tqdm(batched_dataset, desc='Processing behavioral prompts'):
    if steering_type == steering_type.IN_PROMPT:
        batch_tokens_pos = []
        batch_tokens_neg = []
        for idx in range(len(batch_pos)):
            tokens_pos = model.tokenizer.encode(batch_pos[idx], return_tensors="pt")
            tokens_neg = model.tokenizer.encode(batch_neg[idx], return_tensors="pt")
            if len(tokens_pos[0]) != len(tokens_neg[0]) and batch_neg[idx] != "": ##need to even out the lengths
                appstr = " " * abs(len(tokens_neg[0]) - len(tokens_pos[0]))
                apptok = model.tokenizer.encode(appstr, return_tensors="pt")
                if len(tokens_pos[0]) > len(tokens_neg[0]):
                    tokens_neg = torch.cat((tokens_neg, apptok), dim=1)
                else:
                    tokens_pos = torch.cat((tokens_pos, apptok), dim=1)
            batch_tokens_pos.append(tokens_pos)
            batch_tokens_neg.append(tokens_neg)
        batch_tokens_pos = torch.cat(batch_tokens_pos, dim=0)
        batch_tokens_neg = torch.cat(batch_tokens_neg, dim=0)

        get_at = add_at = "start"
    else:
        encoded_pos = model.tokenizer(batch_pos, return_tensors="pt", padding=True)
        encoded_neg = model.tokenizer(batch_neg, return_tensors="pt", padding=True)
        batch_tokens_pos = encoded_pos['input_ids']
        batch_tokens_neg = encoded_neg['input_ids']
        # Calculate the last/key_token_offset token position for each sequence in the batch
        last_token_positions_pos = (encoded_pos['attention_mask'].sum(dim=1) - key_token_offset).tolist()
        last_token_positions_neg = (encoded_neg['attention_mask'].sum(dim=1) - key_token_offset).tolist()

        if min(last_token_positions_pos)<= (max(token_offsets) if token_offsets else priortoks+1) or min(last_token_positions_pos)<= (max(token_offsets) if token_offsets else priortoks+1):
            print("input too short")
            continue

        get_at = add_at = "end"

    layers_positions = {}
    for layer in layers:
        if steering_type == "In prompt": layers_positions[layer] = [list(range(len(batch_tokens_pos[0]))) * len(batch_pos)] 
        else:
            if token_offsets:
                layers_positions[layer] = [[pos-i for i in token_offsets] for pos in last_token_positions_pos]
            else:
                layers_positions[layer] = [[pos-i for i in range(priortoks,-1,-1)] for pos in last_token_positions_pos] #[pos-2, pos-1, pos]

    activations = get_activations(model, batch_tokens_pos, layers_positions, get_at=get_at) #returns a dictionary where keys are layers and values are dicts where keys are positions and values are batchsize d-embed tensors
    for layer, positions in activations.items():
        for pos, tensor in positions.items():#each of these is a stack of batchsize d-embed tensors for a given position
            accumulated_activations_diffs[layer][pos] = torch.cat([accumulated_activations_diffs[layer][pos], tensor.clone()], dim=0)
            accumulated_activations_pos[layer][pos] = torch.cat([accumulated_activations_pos[layer][pos], tensor], dim=0)

    if len(batch_neg[0]) > 1:
        layers_positions = {}
        for layer in layers:
            if steering_type == "In prompt": layers_positions[layer] = [list(range(len(batch_tokens_neg[0]))) * len(batch_pos)] 
            else:
                if token_offsets:
                    layers_positions[layer] = [[pos-i for i in token_offsets] for pos in last_token_positions_neg]
                else:
                    layers_positions[layer] = [[pos-i for i in range(priortoks,-1,-1)] for pos in last_token_positions_neg] #[pos-2, pos-1, pos]
        activations = get_activations(model, batch_tokens_neg, layers_positions, get_at=get_at)
        for layer, positions in activations.items():
            for pos, tensor in positions.items():#each of these is a stack of batchsize d-embed tensors for a given position
                accumulated_activations_neg[layer][pos] = torch.cat([accumulated_activations_neg[layer][pos], tensor], dim=0)
                accumulated_activations_diffs[layer][pos][-len(batch_pos):] -= tensor

meandiffs = {}
for layer, positions in accumulated_activations_pos.items():
    meandiffs[layer] = []
    for pos in range(len(positions)):
        meandiffs[layer].append(torch.mean((accumulated_activations_pos[layer][pos]-accumulated_activations_neg[layer][pos]), dim=0))
normedmeandiffs = {}
for layer, positions in meandiffs.items():
    normedmeandiffs[layer] = []  
    for pos in range(len(positions)):
        normedmeandiffs[layer].append(meandiffs[layer][pos] / torch.norm(meandiffs[layer][pos], p=2, dim=0, keepdim=True))

In [ ]:
### Compute direction vectors from activations to contrastive prompts

agg_type = AggType.MEANDIFF#PCA#
normvec = True
use_raw = True

def get_sign(activations_pos, activations_neg, direction): 
    #decide whether each direction vector is oriented such that "honesty" is high or low, based on a majority vote count of exemplar projections (ie, does the pos example in a pair tend to project higher or lower than the neg example)
    projections_pos = (activations_pos @ direction) / torch.norm(direction)
    projections_neg = (activations_neg @ direction) / torch.norm(direction)

    positive_smaller_mean = np.mean(
        [projections_pos[i] < projections_neg[i] for i in range(len(projections_pos))]
    )
    positive_larger_mean = np.mean(
        [projections_pos[i] > projections_neg[i] for i in range(len(projections_pos))]
    )
    return -1 if positive_smaller_mean > positive_larger_mean else 1

steering_vectors = {} #dictionary where keys are layers and values are lists of n_pos direction tensors
if agg_type == AggType.MEANDIFF: #will also work for simple word/prefix substraction case as in the original steering activation post
    steering_vectors = {}
    
    for layer, positions in accumulated_activations_pos.items():
        steering_vectors[layer] = []
        for pos in range(len(positions)):
            steering_vectors[layer].append((accumulated_activations_pos[layer][pos]-accumulated_activations_neg[layer][pos]).mean(dim=0))
            if normvec:
                steering_vectors[layer][pos] /= torch.norm(steering_vectors[layer][pos], p=2, dim=0, keepdim=True)    
    """
    for layer, positions in accumulated_activations_diffs.items():
        steering_vectors[layer] = []
        for pos in range(len(positions)):
            steering_vectors[layer].append(torch.mean(accumulated_activations_diffs[layer][pos], dim=0))
            if normvec:
                steering_vectors[layer][pos] /= torch.norm(steering_vectors[layer][pos], p=2, dim=0, keepdim=True)  
    """
elif agg_type == AggType.PCA: # get directions for each layer and position using PCA  
    # takes second PC currently, but can take others or weight and combine  
    if use_raw:
        for layer, positions in accumulated_activations_pos.items():#dictionary where keys are layers and values are dicts where keys are positions and values are len(dataset) d-embed tensors
            embeds = []
            for pos in range(len(positions)):
                activations_pos = accumulated_activations_pos[layer][pos]
                activations_neg = accumulated_activations_neg[layer][pos]

                activations = torch.cat([activations_pos, activations_neg], dim=0)
                pca_model = PCA(n_components=2)
                projected_activations = pca_model.fit_transform(activations)#[:,1]
                coef1, coef2 = 0,1#pca_model.explained_variance_[0],pca_model.explained_variance_[1]
                embeds.append(torch.from_numpy(pca_model.components_[0].astype(np.float32))*coef1/(coef1+coef2)*get_sign(accumulated_activations_pos[layer][pos],accumulated_activations_neg[layer][pos],torch.from_numpy(pca_model.components_[0].astype(np.float32))))
                embeds[-1]+=torch.from_numpy(pca_model.components_[1].astype(np.float32))*coef2/(coef1+coef2)*get_sign(accumulated_activations_pos[layer][pos],accumulated_activations_neg[layer][pos],torch.from_numpy(pca_model.components_[1].astype(np.float32)))
            steering_vectors[layer] = torch.stack(embeds)
    else:
        for layer, positions in accumulated_activations_diffs.items():#dictionary where keys are layers and values are dicts where keys are positions and values are len(dataset) d-embed tensors
            embeds = []
            for pos in range(len(positions)):
                train = positions[pos] - positions[pos].mean(axis=0, keepdims=True)
                pca_model = PCA(n_components=2, whiten=False).fit(train)
                coef1, coef2 = 0,1#pca_model.explained_variance_[0],pca_model.explained_variance_[1]
                embeds.append(torch.from_numpy(pca_model.components_[0].astype(np.float32))*coef1/(coef1+coef2)*get_sign(accumulated_activations_pos[layer][pos],accumulated_activations_neg[layer][pos],torch.from_numpy(pca_model.components_[0].astype(np.float32))))
                embeds[-1]+=torch.from_numpy(pca_model.components_[1].astype(np.float32))*coef2/(coef1+coef2)*get_sign(accumulated_activations_pos[layer][pos],accumulated_activations_neg[layer][pos],torch.from_numpy(pca_model.components_[1].astype(np.float32)))
            steering_vectors[layer] = torch.stack(embeds)            
else: print("Unknown AGG_TYPE")


In [ ]:
### Save direction to file 
import pickle
ofname = 'directions_llama3_8bchat_selfrec_hold_18keytokens_meandiff.pkl'
with open(outputdir+ofname, 'wb') as f:
    pickle.dump(steering_vectors, f)

In [ ]:
### Take a look at how the components of the direction vectors are distributed

def plot_activation_distribution(ax, activation, layer):
    ax.hist(activation, bins=50, alpha=1.0)
    ax.set_title(f'Distribution of Activations for Layer {layer}')
    ax.set_xlabel('Activation Value')
    ax.set_ylabel('Frequency')

position = 9
layers_to_show = list(range(model_numlayers))
rows=(len(layers_to_show)-1)//3+1
fig, axes = plt.subplots(rows, 3, figsize=(18, rows*5))
for i, layer in enumerate(layers_to_show):
    plot_activation_distribution(axes[i//3,i%3], nn_probes[layer][position], layer)
#    plot_activation_distribution(axes[i//3,i%3], steering_vectors[layer][position], layer)
#    plot_activation_distribution(axes[i//3,i%3], normedmeandiffs[layer][position], layer)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize differentiation of inputs by vectors: Binary classification: how often does the honest input have higher similarity to the "honesty" direction than the dishonest one does (by definition of the signs, if using training data this will be>=50%)

layers = range(model_numlayers)
position=0
results = {layer: {} for layer in layers}

direction_vector = steering_vectors[layer][position]

for layer in layers:
    sims_pos = (accumulated_activations_pos[layer][position] @ direction_vector) / (torch.norm(direction_vector)*torch.norm(accumulated_activations_pos[layer][position]))
    sims_neg = (accumulated_activations_neg[layer][position] @ direction_vector) / (torch.norm(direction_vector)*torch.norm(accumulated_activations_neg[layer][position]))
    sims = [[sims_pos[i],sims_neg[i]] for i in range(0, len(sims_pos))]
    
    cors = np.mean([max(S) == S[0] for S in sims])
    
    results[layer] = cors

plt.plot(layers, [results[layer] for layer in layers], marker='o')
plt.show()


In [ ]:
### Visualize differentiation: Plot mean projections/cosine sim between inputs and directions

position = 4
projections_honest={}
projections_dishonest={}
cos_sim=True

direction_vector = steering_vectors[layer][position]

for layer in layers:
    pos_vec = accumulated_activations_pos[layer][position]
    neg_vec = accumulated_activations_neg[layer][position]

    projections_honest[layer] = (pos_vec @ direction_vector) / (torch.norm(direction_vector) * (torch.norm(pos_vec, dim=1, keepdim=True) if cos_sim else 1))
    projections_dishonest[layer] = (neg_vec @ direction_vector) / (torch.norm(direction_vector) * (torch.norm(neg_vec, dim=1, keepdim=True) if cos_sim else 1))

projections_honest_means = [torch.mean(projections_honest[layer]).item() for layer in layers] 
projections_dishonest_means = [torch.mean(projections_dishonest[layer]).item() for layer in layers]
ylabel = 'Cosine Similarity to' if cos_sim else 'Projection onto'
plt.figure(figsize=(10, 5))
plt.plot(layers, projections_honest_means, label='Selfrec', marker='o') 
plt.plot(layers, projections_dishonest_means, label='Neutral', marker='x')
plt.xlabel('Layer')
plt.ylabel(f'Mean {ylabel} the "Selfrec" Direction')
plt.title(f'Mean Input {ylabel} "Selfrec" Direction Across Layers')
plt.legend()
plt.show()

In [ ]:
## Plot input data projections onto vectors as scatterplots or histograms
 
ans1="A"#"Y"
ans2="B"#"N"

def plot_projection(activations_pos, activations_neg, pc, ax, title, type):
    
    if type!="both":
        lbl = type.split("_")[1]
        activations_pos_projected = np.dot(activations_pos, pc)
        activations_neg_projected = np.dot(activations_neg, pc)
        ax.hist(activations_pos_projected, bins=50, alpha=0.5, label='+'+lbl)
        ax.hist(activations_neg_projected, bins=50, alpha=0.5, label='-'+lbl)
        ax.legend()

    else:
        activations_pos_projected = np.dot(activations_pos, pc.T)
        activations_neg_projected = np.dot(activations_neg, pc.T)
        for i, (x, y) in enumerate(activations_pos_projected):
            if contrast_task:
                if letters_selfrec[i] == ans1:#Say you wrote it
                    ax.scatter(x, y, color="blue", marker="o", alpha=0.4)
                elif letters_selfrec[i] == ans2:#Say you didn't write it
                    ax.scatter(x, y, color="blue", marker="x", alpha=0.4)
            else:
                if letters_pos[i] == ans1:#True yes
                    ax.scatter(x, y, color="blue", marker="o", alpha=0.4)
                elif letters_pos[i] == ans2:#Rev no#False no
                    ax.scatter(x, y, color="blue", marker="x", alpha=0.4)

        for i, (x, y) in enumerate(activations_neg_projected):
            if contrast_task:
                if letters_neutral[i] == ans1:#Say it has neutral characteristic
                    ax.scatter(x, y, color="red", marker="o", alpha=0.4)
                elif letters_neutral[i] == ans2:#Say it doesn't have neutral characteristic
                    ax.scatter(x, y, color="red", marker="x", alpha=0.4)
            else:
                if letters_neg[i] == ans1:#Rev yes#False yes
                    ax.scatter(x, y, color="red", marker="o", alpha=0.4)
                elif letters_neg[i] == ans2:#True no#False no
                    ax.scatter(x, y, color="red", marker="x", alpha=0.4)

    ax.set_title(title)
    ax.set_xlabel('PC1' if "pc2" not in type else 'PC2')
    if type=="both":
      ax.legend(handles=[
          plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label=f"Pos {behavior} - {ans1}"),
          plt.Line2D([0], [0], marker='x', color='blue', markerfacecolor='blue', markersize=10, label=f"Pos {behavior} - {ans2}"),
          plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label=f"Neg {behavior} - {ans1}"),
          plt.Line2D([0], [0], marker='x', color='red', markerfacecolor='red', markersize=10, label=f"Neg {behavior} - {ans2}")
      ])
    else:
      ax.legend()



In [ ]:
## Show PCA projections for every layer

pcatype = "raw"
position = 10#priortoks - 1
cols = 3
rows = (len(layers) - 1) // cols + 1
fig, axes = plt.subplots(rows, cols, figsize=(18, rows*5))
for i, layer in enumerate(layers):
    activations_pos = accumulated_activations_pos[layer][position]
    activations_neg = accumulated_activations_neg[layer][position]
    
    if pcatype == "raw":
        activations = torch.cat([activations_pos, activations_neg], dim=0)
        pca_input = activations - activations.mean(axis=0, keepdims=True)
    else:            
        pca_input = accumulated_activations_diffs[layer][pos] - accumulated_activations_diffs[layer][pos].mean(axis=0, keepdims=True)
    
    pca_model = PCA(n_components=4, whiten=False).fit(pca_input)
    pc1 = pca_model.components_[0] 
    pc2 = pca_model.components_[1] #steering_vectors[layer][position]#nn_probes[layer]#
    print(pca_model.explained_variance_)
    plot_projection(activations_pos, activations_neg, np.vstack((pc1, pc2)), axes[i // cols, i % cols], f"{behavior}, layer {layer}", type="both")
    
plt.tight_layout()
plt.show()

In [ ]:
# plot histograms of projections of activation data onto direction vector, across layers

def plot_probe_projections(ax, probe, X_pos, X_neg, layer, position, bias=None):
    X = torch.cat([X_pos, X_neg], dim=0)
    y = torch.cat([torch.ones(X_pos.shape[0]), torch.zeros(X_neg.shape[0])])
    projections = X @ probe + (bias if bias else 0)
    
    #ax.scatter(projections[y == 1], [1] * len(projections[y == 1]), color='blue', alpha=0.5, label='Behavior B (Pos)')
    #ax.scatter(projections[y == 0], [0] * len(projections[y == 0]), color='red', alpha=0.5, label='Behavior B\' (Neg)')
    projections_pos = projections[y == 1].detach().numpy()
    projections_neg = projections[y == 0].detach().numpy()
    
    # Calculate combined range for shared bins
    min_value = min(projections_pos.min(), projections_neg.min())
    max_value = max(projections_pos.max(), projections_neg.max())
    bins = np.linspace(min_value, max_value, 20)
    
    ax.hist(projections_pos, bins=bins, alpha=0.5, label='Behavior B', color='blue')
    ax.hist(projections_neg, bins=bins, alpha=0.5, label='Behavior B\'', color='red')
    ax.set_title(f'Histogram of Projections by Linear Probe for Layer {layer}, Position {position}')
    ax.set_ylabel('Frequency')
    ax.set_title(f'Projection of Activations for Layer {layer}, Position {position}')
    ax.set_xlabel('Projection Value')
    ax.legend()

position = 0

cols = 3
rows = (len(layers) - 1) // cols + 1
fig, axes = plt.subplots(rows, cols, figsize=(18, rows*5))
for i, layer in enumerate(nn_probes):
    direction_vector = nn_probes[layer]#nn_probes[layer][position]
#    plot_probe_projections(axes[i // cols, i % cols], direction_vector, accumulated_activations_pos[layer][position], accumulated_activations_neg[layer][position], layer, position)
    plot_probe_projections(axes[i // cols, i % cols], direction_vector, accumulated_activations_pos[layer][position][val_indices], accumulated_activations_neg[layer][position][val_indices], layer, position)
plt.tight_layout()
plt.show()

In [ ]:
### Logit lens visualization of steering vector

import matplotlib.patches as patches

position=5
top_k = 10 
mult=20
colorscale = "Reds" if mult>=0 else "Blues"

token_data = []
probs_data = []

for layer in range(model_numlayers):
    vec=(mult*steering_vectors[layer][position]).to(device)
#    vec=(mult*normedmeandiffs[layer][0]).to(device)
    unembedded = model.lm_head(model.model.norm(vec.to(model.dtype)))
    softmaxed = torch.nn.functional.softmax(unembedded, dim=-1)
    values, indices = torch.topk(softmaxed, top_k)
    probs_percent = [v for v in values.tolist()]
    tokens = model.tokenizer.batch_decode(indices.unsqueeze(-1))
    token_data.append(tokens)
    probs_data.append(probs_percent) 

probs_array = np.array(probs_data)
token_labels = np.array(token_data)

plt.rcParams['font.family'] = 'DejaVu Sans' 
plt.figure(figsize=(14, 8))
ax = sns.heatmap(probs_array, annot=token_labels, fmt='', cmap=colorscale, xticklabels=False, yticklabels=[f"Layer {l}" for l in range(model_numlayers)])
ax.set_title(f"Logit Lens Top Token Probabilities at Multiplier {mult}")
ax.set_xlabel(f"Top {top_k} Tokens")

# Adding a highlight for specific layers
for layerlist in [[17,18]]:
    rect = patches.Rectangle((0, layerlist[0]), top_k, len(layerlist), linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)

plt.show()


In [ ]:
### Capture projections of inputs onto the behavioral direction vector during generation

#import pickle
#fname = 'directions_llama2_13b_f16_persona_lasttoken_pc2raw.pkl'
#with open(outputdir+fname, 'rb') as f:
#    steering_vectors = pickle.load(f)

def cln(text):
    return text.replace(pos_persona,"").replace(neg_persona,"").replace("  "," ")[:-2]
lookback=19
model.tokenizer.padding_side = "left"
sampling_kwargs={"use_cache": True, "pad_token_id": model.tokenizer.eos_token_id, "max_new_tokens": 2}

directions = {layer: posvec[-1] for layer, posvec in steering_vectors.items()} # just look at final position vector for simplicity
###directions = {layer: posvec[-1] for layer, posvec in fact_vectors.items()} # just look at final position vector for simplicity

inputdata = [cln(x) for x, y in learn_directions_dataset] + [cln(y) for x, y in learn_directions_dataset] #first half honesty, second half dishonesty
#inputdata = [x for x, y in learn_directions_dataset[:len(learn_directions_dataset)//2]] + [y for x, y in learn_directions_dataset[:len(learn_directions_dataset)//2]] + [x for x, y in learn_directions_dataset[len(learn_directions_dataset)//2:]] + [y for x, y in learn_directions_dataset[len(learn_directions_dataset)//2:]]#first half "A", second half "B"
#inputdata = [x for x, y in learn_directions_dataset[:len(learn_directions_dataset)//2]] + [y for x, y in learn_directions_dataset[len(learn_directions_dataset)//2:]] + [y for x, y in learn_directions_dataset[:len(learn_directions_dataset)//2]] + [x for x, y in learn_directions_dataset[len(learn_directions_dataset)//2:]] #first half true facts, second half false facts
#inputdata = [prompts_to_be_steered_tqa[i] for i in range(0,len(prompts_to_be_steered_tqa),2)] + [prompts_to_be_steered_tqa[i] for i in range(1,len(prompts_to_be_steered_tqa),2)]
#inputdata = [gpt4facts_prompts[i] for j in range(0, len(gpt4facts_prompts)//10 , 2) for i in range(j*10, j*10+10)] + [gpt4facts_prompts[i] for j in range(1, len(gpt4facts_prompts)//10 , 2) for i in range(j*10, j*10+10)]


layers = range(model_numlayers)

accumulated_projections = defaultdict(lambda: defaultdict(lambda: torch.empty(0)))

batch_size=32
batched_inputs = [
        inputdata[p : p + batch_size] for p in range(0, len(inputdata), batch_size)
    ]

clear_hooks(model)
model.to(device)
for batch in tqdm(batched_inputs, desc='Generating projections'):
    if prepend_bos: batch = [model.tokenizer.bos_token + input for input in batch]
    inputs = model.tokenizer(batch, return_tensors="pt", padding=True)

    get_at = add_at = "end"

    layers_positions = {}
    for layer in layers:
        layers_positions[layer] = [[len(inputs['input_ids'][pos])-offset for offset in range(lookback,0,-1)] for pos in range(len(inputs['input_ids']))]

    activations = get_activations_and_generate(model, inputs, layers_positions, sampling_kwargs, get_at=get_at) #returns a dictionary where keys are layers and values are dicts where keys are positions and values are batchsize d-embed tensors
    for layer, positions in activations.items():
        for pos, tensor in positions.items():#each of these is a stack of batchsize d-embed tensors for a given position
            projection = (tensor @ directions[layer].to(tensor.dtype)) / (torch.norm(directions[layer]) * torch.norm(tensor, dim=1))
            accumulated_projections[layer][pos] = torch.cat([accumulated_projections[layer][pos], projection], dim=0)


In [ ]:
### Visualization: Heatamps out activation projections onto direction vectors across sequence positions and layers
import seaborn as sns

layers = list(range(len(accumulated_projections)))  
positions = range(len(accumulated_projections[0])) 
offset=lookback
split=len(accumulated_projections[0][0])//2
data = np.zeros((len(layers), len(positions)))
honest_mean_data = np.zeros((len(layers), len(positions)))
dishonest_mean_data = np.zeros((len(layers), len(positions)))

for i, layer in enumerate(layers):
    for j, position in enumerate(positions):
        honest_mean_data[i, j] = torch.mean(accumulated_projections[layer][position][:split]).item()
        dishonest_mean_data[i, j] = torch.mean(accumulated_projections[layer][position][split:]).item()
        data[i, j] = honest_mean_data[i, j] - dishonest_mean_data[i, j]

data_flipped = data[::-1, :] # Flip to show first layer at bottom

max_abs_value = np.max(np.abs(data_flipped))
rmabs = round(max_abs_value, -int(np.floor(np.log10(max_abs_value))))
    
plt.figure(figsize=(10, 8))
sns.heatmap(data_flipped, xticklabels=[p - offset for p in positions], yticklabels=sorted(layers,reverse=True), annot=False, cmap='coolwarm', vmin=-rmabs, vmax=rmabs)
plt.xlabel('Position')
plt.ylabel('Layer')
plt.title('Difference in Projections, Honesty - Dishonesty')
plt.show()

data_flipped = honest_mean_data[::-1, :] # Flip to show first layer at bottom

max_abs_value = np.max(np.abs(data_flipped))
rmabs = round(max_abs_value, -int(np.floor(np.log10(max_abs_value))))

plt.figure(figsize=(10, 8))
sns.heatmap(data_flipped, xticklabels=[p - offset for p in positions], yticklabels=sorted(layers,reverse=True), annot=False, cmap='coolwarm', vmin=-rmabs, vmax=rmabs)
plt.xlabel('Position')
plt.ylabel('Layer')
plt.title('Honesty Projections')
plt.show()

data_flipped = dishonest_mean_data[::-1, :] # Flip to show first layer at bottom

max_abs_value = np.max(np.abs(data_flipped))
rmabs = round(max_abs_value, -int(np.floor(np.log10(max_abs_value))))

plt.figure(figsize=(10, 8))
sns.heatmap(data_flipped, xticklabels=[p - offset for p in positions], yticklabels=sorted(layers,reverse=True), annot=False, cmap='coolwarm', vmin=-rmabs, vmax=rmabs)
plt.xlabel('Position')
plt.ylabel('Layer')
plt.title('Dishonesty Projections')
plt.show()